In [1]:
!pip install scrapy -q

In [2]:
import pandas as pd

df = pd.read_json('/content/drive/MyDrive/Jedha_Fullstack/Kayak_Project/Hebergements_urls.json')
df.head(3)

,hebergement,url
0,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...
1,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...
2,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...


In [3]:
#Réduction des urls à leur format minimal

df['url'] = df['url'].apply(lambda x: x.split("&")[0])

In [4]:
#Spider pour scraper les détails de chaque hébergements depuis leur url propre

import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class BookingDetailsSpider(scrapy.Spider):
    # Name of your spider
    name = "BookingDetails"

    # Starting URL
    start_urls = df['url']
    download_delay = 0.5

    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url=url, callback=self.scrape_results)


    def scrape_results(self, response):
        hebergements = response.xpath('//*[@id="hp_hotel_name"]/div/h2/text()')
        cities = response.xpath('//input[@name="ss"]')
        locations =  response.xpath('//*[@id="showMap2"]/span[1]/text()')
        geos = response.xpath('//*[@id="hotel_sidebar_static_map"]')
        notes = response.xpath('//*[@id="js--hp-gallery-scorecard"]/a/div/div/div/div[1]/text()')
        descs = response.xpath('//*[@id="property_description_content"]/div[2]/p/text()')

        for hebergement, city, loc, geo, note, desc in zip(hebergements, cities, locations, geos, notes, descs):

            yield {
            "url" : response.url,
            "hebergement" : hebergement.get(),
            "city" : city.attrib['value'],
            "location" : loc.get(),
            "geos" : geo.attrib["data-atlas-latlng"],
            "note" : note.get(),
            "desc" : desc.get()
            }


filename = "Hebergements_details.json"

if filename in os.listdir('/content/drive/MyDrive/Jedha_Fullstack/Kayak_Project/'):
        os.remove('/content/drive/MyDrive/Jedha_Fullstack/Kayak_Project/' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    'DOWNLOAD_DELAY' : 0.5,
    "FEEDS": {
        '/content/drive/MyDrive/Jedha_Fullstack/Kayak_Project/' + filename: {"format": "json"},
    }
})

process.crawl(BookingDetailsSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.9.0 started (bot: scrapybot)
2023-07-06 07:56:48 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform Linux-5.15.107+-x86_64-with-glibc2.31
2023-07-06 07:56:48 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform Linux-5.15.107+-x86_64-with-glibc2.31
INFO:scrapy.crawler:Overridden settings:
{'DOWNLOAD_DELAY': 0.5, 'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2023-07-06 07:56:48 [scrapy.crawler] INFO: Overridden settings:
{'DOWNLOAD_DELAY': 0.5, 'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}